# 1

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
import plotly.express as px


## 1.1

In [ ]:

df = pd.read_csv('/content/Electric_Production.csv', parse_dates=['DATE'], index_col='DATE')
result= seasonal_decompose(df['IPG2211A2N'], model='additive', period=12)


## 1.2

In [ ]:
fig = make_subplots(rows=4, cols=1, shared_xaxes=True,
                    subplot_titles=('Observed', 'Trend', 'Seasonal', 'Residual'),
                    vertical_spacing=0.05)
fig.add_trace(go.Scatter(x=result.observed.index, y=result.observed, mode='lines', name='Observed', line=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=result.trend.index, y=result.trend, mode='lines', name='Trend', line=dict(color='green')), row=2, col=1)
fig.add_trace(go.Scatter(x=result.seasonal.index, y=result.seasonal, mode='lines', name='Seasonal', line=dict(color='orange')), row=3, col=1)
fig.add_trace(go.Scatter(x=result.resid.index, y=result.resid, mode='lines', name='Residual', line=dict(color='red')), row=4, col=1)
fig.update_layout(height=800, width=900, title_text="Time Series Decomposition", showlegend=False)
fig.show()

## 1.3

In [ ]:

fig = go.Figure()
fig.add_trace(go.Scatter(x=result.observed.index, y=result.observed, mode='lines', name='Observed', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=result.trend.index, y=result.trend, mode='lines', name='Trend', line=dict(color='green', dash='dash')))
fig.add_trace(go.Scatter(x=result.seasonal.index, y=result.seasonal, mode='lines', name='Seasonal', line=dict(color='orange', dash='dot')))
fig.add_trace(go.Scatter(x=result.resid.index, y=result.resid, mode='lines', name='Residual', line=dict(color='red', dash='dashdot')))
fig.update_layout(title='Combined Time Series Components',
                  xaxis_title='Date',
                  yaxis_title='Electric Production',
                  legend_title='Components',
                  height=600, width=900)

fig.show()

## 1.4

In [ ]:

model = ARIMA(df['IPG2211A2N'], order=(5, 1, 0))
model_fit = model.fit()
forecast = model_fit.forecast(steps=12)
forecast_index = pd.date_range(df.index[-1], periods=12, freq='M')
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['IPG2211A2N'], mode='lines', name='Observed', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=forecast_index, y=forecast, mode='lines', name='Forecast', line=dict(color='red', dash='dash')))
fig.update_layout(title='Electric Production Forecast',
                  xaxis_title='Date',
                  yaxis_title='Electric Production',
                  legend_title='Data',
                  height=600, width=900)

fig.show()

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

<ipython-input-5-c645d5b046bd>:4: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



# 2

In [ ]:

data = pd.read_csv('/content/Walmart_Sales.csv')

data['Date'] = pd.to_datetime(data['Date'], infer_datetime_format=True, errors='coerce')



<ipython-input-9-af535ea5077d>:3: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.



In [ ]:
data.set_index('Date', inplace=True)
data.head()

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Unnamed: 8,Unnamed: 9,Unnamed: 10
Date,,,,,,,,,,
2010-02-05,1,1643690.90,0,42.31,2.572,211.096358,8.106,NaN,NaN,20/5/2020
2010-02-12,1,1641957.44,1,38.51,2.548,211.242170,8.106,NaN,NaN,NaN
NaT,1,1611968.17,0,39.93,2.514,211.289143,8.106,NaN,NaN,NaN
NaT,1,1409727.59,0,46.63,2.561,211.319643,8.106,NaN,NaN,NaN
2010-03-05,1,1554806.68,0,46.50,2.625,211.350143,8.106,NaN,NaN,NaN


In [ ]:

weekly_sales = data['Weekly_Sales'].resample('W').sum()
weekly_sales.head()

,Weekly_Sales
Date,
2010-02-07,29666446.67
2010-02-14,28999966.90
2010-02-21,0.00
2010-02-28,0.00
2010-03-07,28222708.26


In [ ]:

fig = px.line(weekly_sales,
              x=weekly_sales.index,
              y=weekly_sales.values,
              title='Weekly Sales Over Time',
              labels={'x': 'Date', 'y': 'Total Weekly Sales'})

fig.show()

In [ ]:

weekly_sales_trend = weekly_sales.rolling(window=52).mean()
fig = px.line(weekly_sales_trend,
              x=weekly_sales_trend.index,
              y=weekly_sales_trend.values,
              title='Weekly Sales Trend (Moving Average)',
              labels={'x': 'Date', 'y': 'Trend of Weekly Sales'})

fig.show()

In [ ]:

decomposition = seasonal_decompose(weekly_sales, model='additive')

# Vẽ chuỗi thời gian mùa vụ
fig = px.line(decomposition.seasonal,
              x=decomposition.seasonal.index,
              y=decomposition.seasonal.values,
              title='Seasonal Component of Weekly Sales',
              labels={'x': 'Date', 'y': 'Seasonality'})

fig.show()

# Vẽ chuỗi thời gian chu kỳ
fig = px.line(decomposition.trend,
              x=decomposition.trend.index,
              y=decomposition.trend.values,
              title='Trend Component of Weekly Sales',
              labels={'x': 'Date', 'y': 'Trend'})

fig.show()

ModuleNotFoundError: No module named 'fbprophet'

In [ ]:
# Tính trung bình giá nhiên liệu theo tuần
fuel_price = data['Fuel_Price'].resample('W').mean()

# Vẽ biểu đồ Fuel_Price
fig = px.line(fuel_price,
              x=fuel_price.index,
              y=fuel_price.values,
              title='Fuel Price Over Time',
              labels={'x': 'Date', 'y': 'Fuel Price'})

fig.show()

NameError: name 'Prophet' is not defined